In [19]:
!pwd

/kaggle/working


In [20]:
!pip install --no-deps '../input/torchlibrosa/torchlibrosa-0.0.4-py3-none-any.whl' 
!pip install ../input/librosa1

Processing /kaggle/input/librosa1
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201374 sha256=0b74e66703d30e0ae06f992650a80ad80622c904f000c4296786b1d390a68d8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-idfjoxt1/wheels/df/fc/c9/6650455770bd14d81771486dde26da9af25f0da4295d195271
Successfully built librosa
  Attempting uninstall: librosa
    Found existing installation: librosa 0.8.0
    Uninstalling librosa-0.8.0:
      Successfully uninstalled librosa-0.8.0


In [21]:
#!pip install librosa
#!pip install torchlibrosa
import sys
from pathlib import Path
sys.path.insert(0,"../input/cnn14-sig")
print(sys.path)
import numpy as np
import pandas as pd
from pathlib import Path
import torch
from fastprogress import progress_bar
from dataset.mp3_dataset import INV_BIRD_CODE
import warnings
from contextlib import contextmanager
from typing import Optional
import logging
import time
import librosa
from models.backbones import  Cnn14_16k
from config.config_Cnn14_16k import model_config
import torch.nn.functional as F

ROOT = Path.cwd().parent
print(ROOT)
RAW_DATA = ROOT / "input/birdsong-recognition"
TEST_AUDIO_DIR = RAW_DATA / "test_audio"
weights_path=ROOT/"input/cnn14-sig/output/best-checkpoint-035epoch.tar"
SR = 32000
if not TEST_AUDIO_DIR.exists():
    TEST_AUDIO_DIR = ROOT/'input' / "birdcall-check" / "test_audio"
    test = pd.read_csv(ROOT/'input'/ "birdcall-check" / "test.csv")
else:
    test = pd.read_csv(RAW_DATA / "test.csv")

['../input/cnn14-sig', '../input/cnn14-sig', '..', '../input/cnn14-sig', '/kaggle/working', '/kaggle/lib/kagglegym', '/kaggle/lib', '/kaggle/input/birdsong-recognition', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '', '/root/.local/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages', '/src/bq-helper', '/opt/conda/lib/python3.7/site-packages/IPython/extensions', '/root/.ipython']
/kaggle


In [22]:
def get_model(config, weights_path):
    model = Cnn14_16k(**config)
    checkpoint = torch.load(weights_path)
    model.load_state_dict(checkpoint["model_state_dict"])
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    return model

In [23]:
def split_clip(length,clip,period,stride,sr):
    clips=[]
    clip=clip[0:int(length*sr)].astype(np.float32)
    start=0
    while start<int(length*sr):
        y=clip[start:start+period*sr]
        if len(y)!=period*sr:
            y_pad = np.zeros(period * sr, dtype=np.float32)
            y_pad[0:len(y)] = y
            clips.append(y_pad)
            break
        start=start+stride*sr
        clips.append(y)
    return clips 


In [24]:
def rand_split_clip(rand_len,length,clip,period,stride,sr):
    clips=[]
    clip=clip[0:int(length*sr)].astype(np.float32)
    start = np.random.randint(length*sr - rand_len*sr)
    end=int(start+rand_len*sr)
    while start<end:
        y=clip[start:start+period*sr]
        if len(y)!=period*sr:
            y_pad = np.zeros(period * sr, dtype=np.float32)
            y_pad[0:len(y)] = y
            clips.append(y_pad)
            break
        start=start+stride*sr
        clips.append(y)
    return clips 

In [25]:
def prediction_for_clip(test_df, clip,model,threshold=0.5,period=5,stride=2,sr=32000,cut_length=30):
    prediction_dict = {}
    for i in range(test_df.shape[0]):
        temp=test_df.loc[i, :]
        row_id = temp.row_id
        seconds=temp.seconds
        if np.isnan(seconds) :
            seconds=int(len(clip)/sr)
        if seconds<=cut_length:
            splited=split_clip(seconds, clip, period, stride, sr)
        else:
            splited=rand_split_clip(cut_length,seconds, clip, period, stride, sr)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        all_events = set()
        audio=torch.tensor(splited).to(device).float()
        with torch.no_grad():
            prediction = model(audio)["clipwise_output"]
            proba = prediction.detach().cpu().numpy()                  
        labels = proba >= threshold
        for label in labels:
            label=np.argwhere(label).reshape(-1).tolist()
            for i in label: 
                all_events.add(i)
        if len(all_events) == 0:
            prediction_dict[row_id] = "nocall"
        else:
            labels_str_list = list(map(lambda x: INV_BIRD_CODE[x], all_events))
            label_string = " ".join(labels_str_list)
            prediction_dict[row_id] = label_string
    return prediction_dict


In [26]:
def prediction(test_df,test_audio,model_config,target_sr,threshold,period,stride,weights_path,cut_length):
    model = get_model(model_config,weights_path)
    unique_audio_id = test_df.audio_id.unique()
    warnings.filterwarnings("ignore")
    prediction_dfs = []
    for audio_id in unique_audio_id:
        clip, _ = librosa.load(test_audio / (audio_id + ".mp3"),sr=target_sr,mono=True,res_type="kaiser_fast")      
        test_df_for_audio_id = test_df.query(f"audio_id == '{audio_id}'").reset_index(drop=True)
        prediction_dict = prediction_for_clip(test_df_for_audio_id,clip,model,threshold,period,stride,target_sr,cut_length)
        row_id = list(prediction_dict.keys())
        birds = list(prediction_dict.values())
        prediction_df = pd.DataFrame({"row_id": row_id,"birds": birds})
        prediction_dfs.append(prediction_df)
    prediction_df = pd.concat(prediction_dfs, axis=0, sort=False).reset_index(drop=True)
    return prediction_df



In [ ]:
submission=prediction(test_df=test,
                        test_audio=TEST_AUDIO_DIR,
                        model_config=model_config,
                        target_sr=SR,
                        threshold=0.6,
                        period=5,
                        stride=5,
                        weights_path=weights_path,
                        cut_length=30
                        )
submission.to_csv("submission.csv", index=False)
print(submission)


5.0
10.0
15.0
20.0
25.0
5.0
10.0
15.0
20.0
25.0
30.0
35.0
5.0
10.0
15.0
20.0
25.0
30.0
35.0
5.0
10.0
15.0
20.0
25.0
30.0
5.0
10.0
15.0
20.0
25.0
30.0
35.0
5.0
5.0
10.0
15.0
20.0
25.0
30.0
35.0
40.0
45.0
5.0
10.0
15.0
20.0
25.0
